# Memory Store Demo

# 1. 初始化


In [2]:

from tablestore_for_agent_memory.base.common import MetaType
from tablestore_for_agent_memory.memory.memory_store import MemoryStore
import os

endpoint = os.getenv("tablestore_end_point")
instance_name = os.getenv("tablestore_instance_name")
access_key_id = os.getenv("tablestore_access_key_id")
access_key_secret = os.getenv("tablestore_access_key_secret")

# 根据 Session 的更新时间 进行 list_recent_sessions 时候，需要返回哪些字段?
session_secondary_index_meta = {
    "meta_string": MetaType.STRING,
    "meta_long": MetaType.INTEGER,
    "meta_double": MetaType.DOUBLE,
    "meta_boolean": MetaType.BOOLEAN,
    "meta_bytes": MetaType.BINARY,
}

memory_store = MemoryStore(
    endpoint=endpoint,
    instance_name=instance_name,
    access_key_id=access_key_id,
    access_key_secret=access_key_secret,
    session_secondary_index_meta=session_secondary_index_meta,
)

In [5]:
# 创建表，仅需执行一次
memory_store.init_table()

In [4]:
# 删除表（内部接口，仅供测试使用）
memory_store._delete_table()

# 2. Session 会话管理

一个Session属于一个User，一个User有n个Session，每个Session有n个聊天消息Message，本章节主要介绍Session的管理

## 2.1 声明 Session

In [35]:
from tablestore_for_agent_memory.base.common import microseconds_timestamp
from tablestore_for_agent_memory.base.base_memory_store import Session

# 声明一个Session（user_id和session_id联合起来确认唯一一个session）,推荐使用uuid当做session_id
session = Session(user_id="1", session_id="2")
# 指定Session的更新时间(可以不写，默认是微妙时间戳)
session.update_time = microseconds_timestamp()
# 给这个Session附加一些meta信息,支持的类型包括以下:
session.metadata["meta_string"] = "4"
session.metadata["meta_long"] = 5
session.metadata["meta_double"] = 6.6
session.metadata["meta_boolean"] = True
session.metadata["meta_bytes"] = bytearray("China", encoding="utf8")

## 2.2 创建、更新、查询、删除 Session


In [36]:
# 覆盖写
memory_store.put_session(session)
# 查询
session = memory_store.get_session(user_id="1", session_id="2")
print(session)

Session(user_id='1', session_id='2', update_time=1743669966419601, metadata={'meta_boolean': True, 'meta_bytes': bytearray(b'China'), 'meta_double': 6.6, 'meta_long': 5, 'meta_string': '4'})


In [37]:
session.metadata["meta_string"] = "updated"
# 增量更新(不存在则创建新的，普通写入推荐使用put_session)
memory_store.update_session(session)
print(memory_store.get_session(user_id="1", session_id="2"))

Session(user_id='1', session_id='2', update_time=1743669966419601, metadata={'meta_boolean': True, 'meta_bytes': bytearray(b'China'), 'meta_double': 6.6, 'meta_long': 5, 'meta_string': 'updated'})


In [38]:
# 删除
memory_store.delete_session(user_id="1", session_id="2")
print(memory_store.get_session(user_id="1", session_id="2"))

None


## 2.3 展示最近活跃的Session

根据 session 更新时间`session.update_time` 展示最近活跃列表。

### 2.3.1 写入样例数据
 

In [39]:
from faker import Faker
import random

fk_data = Faker(locale="zh_CN")


def random_session(user_id=fk_data.user_name()) -> Session:
    session = Session(user_id=user_id, session_id=fk_data.uuid4())
    session.metadata["meta_string"] = fk_data.name()
    session.metadata["meta_long"] = random.randint(0, 9999999999999)
    session.metadata["meta_double"] = random.uniform(0.0, 1.0)
    session.metadata["meta_boolean"] = random.choice([True, False])
    session.metadata["meta_bytes"] = bytearray(fk_data.name(), encoding="utf8")
    return session


total_count = 100
for i in range(total_count):
    user_id = random.choice(["1", "2"])
    session = random_session(user_id=user_id)
    session.update_time = random.randint(1, 100)
    memory_store.put_session(session)

### 2.3.2 展示某一个用户最近的活跃会话列表


In [40]:
# 接口返回 iterator 迭代器
iterator = memory_store.list_recent_sessions(user_id="1")
for session in iterator:
    print(session)

Session(user_id='1', session_id='推荐使用uuid当做session_id', update_time=1743669948613935, metadata={'meta_boolean': True, 'meta_bytes': bytearray(b'China'), 'meta_double': 6.6, 'meta_long': 5, 'meta_string': '4'})
Session(user_id='1', session_id='5ac16f06-2401-4666-ba7a-70ba8224c68e', update_time=99, metadata={'meta_boolean': False, 'meta_bytes': bytearray(b'\xe5\xbe\x90\xe7\xa7\x80\xe8\x8d\xa3'), 'meta_double': 0.9393316965655581, 'meta_long': 3741995364619, 'meta_string': '方雷'})
Session(user_id='1', session_id='bf449062-4bab-4282-a492-3ed79e93cdca', update_time=96, metadata={'meta_boolean': False, 'meta_bytes': bytearray(b'\xe7\xa8\x8b\xe5\xbd\xac'), 'meta_double': 0.9917839876741434, 'meta_long': 5058771419374, 'meta_string': '宋建'})
Session(user_id='1', session_id='b66197b0-a942-4311-8e98-93a1b103f613', update_time=95, metadata={'meta_boolean': True, 'meta_bytes': bytearray(b'\xe6\x9f\xaf\xe7\xa7\x80\xe5\x85\xb0'), 'meta_double': 0.40591860798588697, 'meta_long': 613451448049, 'meta_str

In [41]:

# 将 iterator 迭代器转成list使用
sessions = list(memory_store.list_recent_sessions(user_id="1"))
print(len(sessions))

# 仅获取其前 5 个值。推荐使用该方法，因为用户量大了，列出所有session会越来越慢，因此仅需列出其最活跃的即可。
sessions = list(memory_store.list_recent_sessions(user_id="1", max_count=5))
print(len(sessions))

# 获取前 5 个
sessions = list(memory_store.list_recent_sessions(user_id="1", max_count=5))
print(len(sessions))

45
5
5


### 2.3.3 根据 metadata 进行 filter

In [42]:
from tablestore_for_agent_memory.base.filter import Filters

# 使用 单个 Filter 进行过滤
sessions = list(memory_store.list_recent_sessions(user_id="1", metadata_filter=Filters.eq("meta_boolean", True)))
print(len(sessions))

25


In [43]:
from tablestore_for_agent_memory.base.filter import Filters

# 使用 多个 Filter 进行过滤
sessions = list(
    memory_store.list_recent_sessions(
        user_id="1",
        metadata_filter=Filters.logical_and([
            Filters.gt("meta_double", 0.5),
            Filters.eq("meta_boolean", True)
        ]),
    )
)
print(len(sessions))

8


### 2.3.4 设置查询Session的截止时间

In [44]:
sessions = list(
    memory_store.list_recent_sessions(
        user_id="1",
        inclusive_end_update_time=50,
        metadata_filter=Filters.eq("meta_boolean", True),
    )
)
print(len(sessions))
print([item.update_time for item in sessions])

15
[1743669948613935, 95, 94, 90, 89, 87, 87, 85, 85, 77, 70, 69, 51, 51, 51]


## 2.4 其它不常用接口


In [51]:
# 所有用户的所有session（注意，量可能较大，请使用迭代器访问，不要放到一个list里）
iterator = memory_store.list_all_sessions()
for session in iterator:
    pass

In [ ]:
# 删除用户的所有session（注意：高危）
memory_store.delete_all_sessions()

# 3. Message 聊天消息

一个Session有n个聊天消息Message，本章节主要介绍Message的使用。

## 3.1 声明 Message


In [45]:
from tablestore_for_agent_memory.base.base_memory_store import Message

# 创建一条message（session_id 和 message_id 联合起来确认唯一一行数据）
message = Message(
    session_id="推荐uuid当做session_id",
    message_id="可以使用uuid或其他业务string当做message_id",
    create_time=random.randint(0, 999999999)  # create_time 不可变，用来确认这条消息在一个会话(session)里的位置信息(或者叫排序信息)
)

message.content = fk_data.text(20)
message.metadata["meta_string"] = fk_data.name_male()
message.metadata["meta_long"] = random.randint(0, 999999999)
message.metadata["meta_double"] = random.uniform(1.0, 2.0)
message.metadata["meta_boolean"] = random.choice([True, False])
message.metadata["meta_bytes"] = bytearray(fk_data.city_name(), encoding="utf8")

## 3.2 写入、查询、更新、删除一条聊天记录

In [46]:
# 写入
memory_store.put_message(message)

# 查询 (不指定create_time)
message_read = memory_store.get_message(session_id=message.session_id, message_id=message.message_id)
print(message_read)

# 查询 (指定create_time，性能更好，查单条message场景的QPS较高的用户可以传。因该领域大多数业务查询单条聊天消息的场景或qps很少，所以不带也没关系，大多数用户的场景是list出来一批最近的聊天消息，而不是查询单条)
message_read = memory_store.get_message(session_id=message.session_id, message_id=message.message_id, create_time=message.create_time)
print(message_read)

# 更新
message.content = "update to: 123"
memory_store.update_message(message)
message_read_after_update = memory_store.get_message(session_id=message.session_id, message_id=message.message_id, create_time=message.create_time)
print(message_read_after_update)

# 删除 (不指定create_time)
memory_store.delete_message(session_id=message.session_id, message_id=message.message_id)
# 删除（(指定create_time，性能更好，原因参考上述查询接口描述）
memory_store.delete_message(session_id=message.session_id, message_id=message.message_id, create_time=message.create_time)
# 删除后查询不到
print(memory_store.get_message(session_id=message.session_id, message_id=message.message_id))



Message(session_id='推荐uuid当做session_id', message_id='可以使用uuid或其他业务string当做message_id', create_time=204946059, content='最大朋友文章文件具有部门政府学生问题.', metadata={'meta_boolean': True, 'meta_bytes': bytearray(b'\xe6\x98\x86\xe6\x98\x8e'), 'meta_double': 1.9910773903013648, 'meta_long': 673673892, 'meta_string': '王帆'})
Message(session_id='推荐uuid当做session_id', message_id='可以使用uuid或其他业务string当做message_id', create_time=204946059, content='最大朋友文章文件具有部门政府学生问题.', metadata={'meta_boolean': True, 'meta_bytes': bytearray(b'\xe6\x98\x86\xe6\x98\x8e'), 'meta_double': 1.9910773903013648, 'meta_long': 673673892, 'meta_string': '王帆'})
Message(session_id='推荐uuid当做session_id', message_id='可以使用uuid或其他业务string当做message_id', create_time=204946059, content='update to: 123', metadata={'meta_boolean': True, 'meta_bytes': bytearray(b'\xe6\x98\x86\xe6\x98\x8e'), 'meta_double': 1.9910773903013648, 'meta_long': 673673892, 'meta_string': '王帆'})
None


## 3.3 获取某一个session会话的聊天消息

### 3.3.1 写入样例数据

In [47]:
from faker import Faker
import random

fk_data = Faker(locale="zh_CN")


def random_message(session_id=fk_data.user_name()) -> Message:
    message = Message(session_id=session_id, message_id=fk_data.uuid4())
    message.create_time = random.randint(0, 999999999)
    if random.choice([True, False]):
        message.content = fk_data.text(20)
    message.metadata["meta_string"] = fk_data.name_male()
    message.metadata["meta_long"] = random.randint(0, 999999999)
    message.metadata["meta_double"] = random.uniform(1.0, 2.0)
    message.metadata["meta_boolean"] = random.choice([True, False])
    message.metadata["meta_bytes"] = bytearray(fk_data.city_name(), encoding="utf8")
    return message


total_count = 50
for i in range(total_count):
    session_id = random.choice(["1", "2"])
    message = random_message(session_id=session_id)
    message.create_time = random.randint(1, 100)
    memory_store.put_message(message)


### 3.3.2 获取所有消息
根据业务方的经验，在大模型聊天场景下，越早的消息其实不太需要的，仅需要最近的个别几条即可，因此更推荐下一小章节的“获取最近的部分消息”。

In [48]:
# 获取某一个session_id的全部聊天消息（默认根据 create_time 从最新到旧排序，接口返回 iterator 迭代器）
iterator = memory_store.get_messages(session_id="1")
# 将 iterator 迭代器 转为list使用
messages = list(iterator)
print(len(messages))

25


In [49]:
# 根据 metadata Filter 获取某一个session的全部数据
messages = list(
    memory_store.get_messages(
        session_id="1",
        metadata_filter=Filters.logical_and([Filters.gt("meta_double", 0.5), Filters.eq("meta_boolean", True)]),
    )
)
print(len(messages))

12


In [50]:
# 所有session的所有message（注意，量可能较大，请使用迭代器访问，不要放到一个list里）
iterator = memory_store.get_all_messages()

### 3.3.3 获取最近的部分消息
#### 3.3.3.1 获取最近的 3 条消息

In [51]:

iterator = memory_store.get_messages(session_id="1", max_count=3)
messages = list(iterator)
print(len(messages))

# 根据 metadata 过滤，返回最近的3条消息
messages = list(memory_store.get_messages(session_id="1", metadata_filter=Filters.eq("meta_boolean", True), max_count=3))
print(len(messages))

3
3



#### 3.3.3.2 按照时间获取


In [52]:

from tablestore_for_agent_memory.base.common import Order

# 获取 session_id=“1” 且创建时间在10~50范围内的数据，排序方式为正序(create_time 从最旧到新)
messages = list(
    memory_store.get_messages(
        session_id="1",
        inclusive_start_create_time=10,
        inclusive_end_create_time=50,
        order=Order.ASC,
    )
)
print(len(messages))

# 获取 session_id=“1” 且创建时间在10~80范围内的数据，排序方式为逆序(create_time 从最新到旧)，因此起始时间是80，结束时间是10.
messages = list(
    memory_store.get_messages(
        session_id="1",
        inclusive_start_create_time=80,
        inclusive_end_create_time=10,
        order=Order.DESC,
    )
)
print(len(messages))

# 获取 session_id=“1” 且创建时间在10~80范围内的数据，排序方式为逆序(create_time 从最新到旧),同时指定 metadata filter
messages = list(
    memory_store.get_messages(
        session_id="1",
        inclusive_start_create_time=80,
        inclusive_end_create_time=10,
        order=Order.DESC,
        metadata_filter=Filters.logical_and([Filters.gt("meta_double", 0.5), Filters.eq("meta_boolean", True)]),
    )
)
print(len(messages))

9
20
9


## 3.4 批量删除 message

In [ ]:
# 删除一个session的全部message
memory_store.delete_messages(session_id="1")

In [ ]:
# 删除全部session的全部message（注意：高危）
memory_store.delete_all_messages()

# 4. 实践

按照用户和大模型交互的流程，将session和message进行实战应用。

## 4.1 新用户进来

In [ ]:
# 创建一个session
session = Session(user_id="1", session_id="session_id_1")
session.update_time = microseconds_timestamp()
session.metadata["meta_使用哪个模型"] = "qwen.5"
memory_store.put_message(session)

# 用户提问：你好，帮我讲个笑话
message = Message(
    session_id="session_id_1",
    message_id="message_id_1",
    create_time=microseconds_timestamp()
)
message.content = "你好，帮我讲个笑话"
message.metadata["meta_访问来源"] = "web"
message.metadata["meta_消息分类"] = "用户"
memory_store.put_message(message) # 记录用户消息
session.update_time = microseconds_timestamp()
memory_store.put_message(session) # 记录用户消息时候，需要同时更新session信息，这里仅以更新update_time为例。

# 大模型返回：小白＋小白=? 小白兔(two)
message = Message(
    session_id="session_id_1",
    message_id="message_id_2", # 消息id改变
    create_time=microseconds_timestamp()
)
message.content = "小白＋小白=? 小白兔(two)"
message.metadata["meta_消息分类"] = "大模型"
memory_store.put_message(message) # 记录大模型消息

# 用户提问：再来一个
message = Message(
    session_id="session_id_1",
    message_id="message_id_3",  # 消息id改变
    create_time=microseconds_timestamp()
)
message.content = "再来一个"
message.metadata["meta_访问来源"] = "web"
message.metadata["meta_消息分类"] = "用户"
memory_store.put_message(message) 
session.update_time = microseconds_timestamp()
memory_store.put_message(session) # 记录用户消息时候，需要同时更新session信息，这里仅以更新update_time为例。
# 查出来上下文消息，也告诉大模型，这样大模型才知道“”
related_messages = list(memory_store.get_messages(session_id="session_id_1", max_count=3))

# 大模型返回：有一个躲猫猫社团，他们团长现在还没找到。
message = Message(
    session_id="session_id_1",
    message_id="message_id_4", # 消息id改变
    create_time=microseconds_timestamp()
)
message.content = "小白＋小白=? 小白兔(two)"
message.metadata["meta_消息分类"] = "大模型"
memory_store.put_message(message) # 记录大模型消息



## 4.2 老用户继续使用历史会话

In [ ]:
# 展示出有哪些历史会话
sessions = list(memory_store.list_recent_sessions(user_id="1", max_count=5))

# 用户点击某一个会话session，这时候可以查询这个session详细信息(因为list最近会话session，有可能meta不全,如有必要刻意考虑再拿一次完整信息)
session = memory_store.get_session(user_id="1", session_id="session_id_1")

# 查出来当前session所有消息进行展示
all_messages = memory_store.get_messages(session_id="session_id_1")

# 后续继续与大模型提问交互即可
